# Efficiently run SetFit Models with Optimum

In [1]:
# Run this notebook using `numactl`` to gain better control on which resources you are running. 
# It is best to run on a single socket.

# NUM_THREADS=<NUM_AVAILABLE_CORES>
# OMP_NUM_THREADS=$NUM_THREADS numactl -C 0-"$(($NUM_THREADS-1))" -m 0 jupyter notebook

In [2]:
!python -m pip install matplotlib -qqq

In [3]:
from pathlib import Path
from time import perf_counter
import os

import evaluate
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
test_dataset = load_dataset("SetFit/sst2")["validation"]
from setfit import SetFitModel
import json
import warnings
warnings.filterwarnings("ignore")

metric = evaluate.load("accuracy")

os.environ["TOKENIZERS_PARALLELISM"] = "false"

class PerformanceBenchmark:
    def __init__(self, model, dataset, optim_type):
        self.model = model
        self.dataset = dataset
        self.optim_type = optim_type
        self.enable_autocast = 'optimum' in self.optim_type.lower()
        
    def compute_accuracy(self):
        with torch.cpu.amp.autocast(enabled=self.enable_autocast):
            preds = self.model.predict(self.dataset["text"])
        labels = self.dataset["label"]
        accuracy = metric.compute(predictions=preds, references=labels)
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
        return accuracy

    def compute_size(self):
        state_dict = self.model.model_body.state_dict()
        tmp_path = Path("model.pt")
        torch.save(state_dict, tmp_path)
        # Calculate size in megabytes
        size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
        # Delete temporary file
        tmp_path.unlink()
        print(f"Model size (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}

    def compute_latency(self, query="that loves its characters and communicates something rather beautiful about human nature"):
        latencies = []
        # Warmup
        for _ in range(10):
            with torch.no_grad(), torch.cpu.amp.autocast(enabled=self.enable_autocast):
                _ = self.model([query])
        # Timed run
        for _ in range(200):
            start_time = perf_counter()
            with torch.no_grad(), torch.cpu.amp.autocast(enabled=self.enable_autocast):
                _ = self.model([query])
            latency = perf_counter() - start_time
            latencies.append(latency)
        # Compute run statistics
        time_avg_ms = 1000 * np.mean(latencies)
        time_std_ms = 1000 * np.std(latencies)
        print(rf"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
        return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}

    def compute_throughput(self, batch_sizes=[64, 512, 8192]):
        results = {}
        num_iters = 5
        replications = int((max(batch_sizes) * (num_iters + 1)) / len(self.dataset))
        replicated_ds = (self.dataset["text"] * replications)[:max(batch_sizes) * num_iters]

        s = ""
        for batch_size in batch_sizes:
            print(f"Batch size: {batch_size}")
            
            throughputs = []
            dataloader = iter(DataLoader(replicated_ds, batch_size=batch_size))
            # Warmup
            for _ in range(2):
                with torch.no_grad(), torch.cpu.amp.autocast(enabled=self.enable_autocast):
                    _ = self.model(next(dataloader))

            dataloader = iter(DataLoader(replicated_ds, batch_size=batch_size))
            # Timed run
            num_loops = num_iters * (max(batch_sizes) / batch_size)
            assert int(num_loops) == num_loops
            
            for _ in range(int(num_loops)):
                start_time = perf_counter()
                with torch.no_grad(), torch.cpu.amp.autocast(enabled=self.enable_autocast):
                    _ = self.model(next(dataloader))
                latency =  perf_counter() - start_time
                throughputs.append(batch_size / latency)
                
            # Compute run statistics
            avg_throughput = np.mean(throughputs)
            std_throughput = np.std(throughputs)
            print(rf"Average throughput (bs={batch_size}) (samples/second): {avg_throughput:.2f} +\- {std_throughput:.2f}\n")
            results[f"throughput_avg_{batch_size}"] = avg_throughput
            results[f"throughput_std_{batch_size}"] = std_throughput
        return results
        
    def run_benchmark(self):
        metrics = {self.optim_type: {}}
        metrics[self.optim_type].update(self.compute_size())
        metrics[self.optim_type].update(self.compute_accuracy())
        metrics[self.optim_type].update(self.compute_latency())
        metrics[self.optim_type].update(self.compute_throughput())
        return metrics

/storage/sdp/anaconda3/envs/setfit-pr/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [4]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_metrics(perf_metrics):
    df = pd.DataFrame.from_dict(perf_metrics, orient="index")

    for idx in df.index:
        df_opt = df.loc[idx]
        plt.errorbar(
            df_opt["time_avg_ms"],
            df_opt["accuracy"] * 100,
            xerr=df_opt["time_std_ms"],
            fmt="o",
            alpha=0.5,
            ms=df_opt["size_mb"] / 15,
            label=idx,
            capsize=5,
            capthick=1,
        )

    legend = plt.legend(loc="lower right")

    plt.ylim(63, 95)
    # Use the slowest model to define the x-axis range
    xlim = max([metrics["time_avg_ms"] for metrics in perf_metrics.values()]) * 1.3
    plt.xlim(0, xlim)
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Average latency with batch_size=1 (ms)")
    plt.show()

In [ ]:
small_model = SetFitModel.from_pretrained("moshew/bge-small-en-v1.5_setfit-sst2-english")
pb = PerformanceBenchmark(model=small_model, dataset=test_dataset, optim_type="bge-small (PyTorch)")
perf_metrics = pb.run_benchmark()
with open("perf_metrics.json", "w") as f:
    json.dump(perf_metrics, f)
plot_metrics(perf_metrics)

Model size (MB) - 127.32
Accuracy on test set - 0.906
Average latency (ms) - 8.74 +\- 0.28
Batch size: 64


## 4. Optimizing with [`optimum-intel`](https://github.com/huggingface/optimum-intel)

In [ ]:
# Install intel-optimum and dependencies
# !python -m pip install torch==2.1 --index-url https://download.pytorch.org/whl/cpu -qqq
# !python -m pip install intel-extension-for-pytorch -qqq
# !python -m pip install intel-openmp -qqq
# !python -m pip install optimum[neural-compressor] --upgrade-strategy eager -qqq
# !python -m pip install mteb simple-parsing -qqq

# !cd ~ && git clone https://github.com/IntelLabs/fastRAG.git && cd -

# RESTART KERNEL AND RUN CELL [2] AGAIN
import os
os._exit(00)

In [ ]:
# Quantize our SetFit model using 100 samples:

QUANIIZTER="~/fastRAG/scripts/optimizations/embedders/quantize_embedder.py"
!python $QUANIIZTER --quantize --model_name "moshew/bge-small-en-v1.5_setfit-sst2-english" --output_path bge-small-en-v1.5_setfit-sst2-quant/ \
    --sample_size 100

In [ ]:
from setfit.exporters.utils import mean_pooling

class INCSetFitModel:
    def __init__(self, inc_model, tokenizer, model_head):
        self.inc_model = inc_model
        self.tokenizer = tokenizer
        self.model_head = model_head

    def predict(self, inputs):
        encoded_inputs = self.tokenizer(
            inputs, padding=True, truncation=True, return_tensors="pt"
        ).to(self.inc_model.device)

        outputs = self.inc_model(**encoded_inputs)
        embeddings = mean_pooling(
            outputs["last_hidden_state"], encoded_inputs["attention_mask"]
        )
        return self.model_head.predict(embeddings.cpu())

    def __call__(self, inputs):
        return self.predict(inputs)

In [ ]:
from optimum.intel import INCModel
from transformers import AutoTokenizer
from setfit import SetFitModel
from datasets import load_dataset
test_dataset = load_dataset("SetFit/sst2")["validation"]
import json
import warnings
warnings.filterwarnings("ignore")

inc_model_path = "bge-small-en-v1.5_setfit-sst2-quant"
tokenizer = AutoTokenizer.from_pretrained(inc_model_path, model_max_length=512)
inc_model = INCModel.from_pretrained(inc_model_path)
model = SetFitModel.from_pretrained("moshew/bge-small-en-v1.5_setfit-sst2-english")
inc_setfit_model = INCSetFitModel(inc_model, tokenizer, model.model_head)

# Perform inference
inc_setfit_model(test_dataset["text"][:5])

In [ ]:
class INCPerformanceBenchmark(PerformanceBenchmark):
    def __init__(self, *args, model_path, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_path = model_path

    def compute_size(self):
        size_mb = Path(self.model_path).stat().st_size / (1024 * 1024)
        print(f"Model size (MB) - {size_mb:.3f}")
        return {"size_mb": size_mb}
        
pb = INCPerformanceBenchmark(
    inc_setfit_model,
    test_dataset,
    "bge-small (optimum-intel)",
    model_path=inc_model_path,
)

with open("perf_metrics.json") as f:
    perf_metrics = json.load(f)
perf_metrics.update(pb.run_benchmark())

for optim_type, results in perf_metrics.items():
    s = [f"{k}: {v:.3f}" for k, v in results.items()]
    print(f"{optim_type}:\n" + "\n".join(s) + "\n")
    
plot_metrics(perf_metrics)

def get_latency_speedup(model_name):
    speedup = perf_metrics['bge-small (PyTorch)']['time_avg_ms'] / perf_metrics[model_name]['time_avg_ms']
    return speedup
print(f"Latency speedup for 'bge-small (optimum-intel)': {get_latency_speedup('bge-small (optimum-intel)'):.2f}x")